In [1]:
import pandas 
print('Pandas version',pandas.__version__)
import numpy
print('Numpy version ',numpy.__version__)
import xgboost
print('XGBOOST version ',xgboost.__version__)
import sklearn
print('Sklearn version ',sklearn.__version__)

Pandas version 1.0.3
Numpy version  1.18.4
XGBOOST version  1.1.1
Sklearn version  0.23.1


In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import *
from sklearn.model_selection import *

In [2]:
train_path       = r'D:\av_hackathons\machine_hack_13\Train.csv'
test_path        = r'D:\av_hackathons\machine_hack_13\Test.csv'
submis_path      = r'D:\av_hackathons\machine_hack_13\Sample_Submission.csv'

train_data       = pd.read_csv(train_path)
test_data        = pd.read_csv(test_path)
train_data['new_col1'] = train_data['AP']/train_data['V']
test_data['new_col1']  = test_data['AP']/test_data['V']

train_y               = train_data['PE'].values

train_data            = train_data.drop(['PE'],axis=1)
train                 = train_data.values
test                  = test_data.values
print(train.shape,test_data.shape)

(9568, 5) (38272, 5)


In [3]:
oof_pred               = np.zeros((len(train), ))
y_pred_1               = np.zeros((len(test), ))
y_pred_2               = np.zeros((len(test), ))
y_pred_3               = np.zeros((len(test), ))
y_pred_4               = np.zeros((len(test), ))
num_models             = 4

n_splits               = 100
error                  = []
kf                     = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (tr_ind, val_ind) in enumerate(kf.split(train, train_y)):
    wghts = [0]*num_models
    rmse  = []
    X_train, X_val     = train[tr_ind], train[val_ind]
    y_train, y_val     = train_y[tr_ind], train_y[val_ind]
    
    model1 = XGBRegressor(n_estimators=400,random_state=42,subsample=0.8)
    model1.fit(X_train,np.log(y_train))
    val_pred1 = model1.predict(X_val)
    rmse.append(np.sqrt(mean_squared_error(y_val, np.exp(val_pred1))))
    y_pred_1  += model1.predict(test) / (n_splits)
    
    model2 = XGBRegressor(n_estimators=500,random_state=42,subsample=0.9)
    model2.fit(X_train,np.log(y_train))
    val_pred2 = model2.predict(X_val)
    rmse.append(np.sqrt(mean_squared_error(y_val, np.exp(val_pred2))))
    y_pred_2  += model2.predict(test) / (n_splits)
    
    model3 = XGBRegressor(n_estimators=600,random_state=42,subsample=0.7)
    model3.fit(X_train,np.log(y_train))
    val_pred3 = model3.predict(X_val)
    rmse.append(np.sqrt(mean_squared_error(y_val, np.exp(val_pred3))))
    y_pred_3  += model3.predict(test) / (n_splits)
    
    model4     = XGBRegressor(n_estimators=700,random_state=42,subsample=0.7,reg_lambda=2)
    model4.fit(X_train,np.log(y_train))
    val_pred4  = model4.predict(X_val)
    rmse.append(np.sqrt(mean_squared_error(y_val, np.exp(val_pred4))))
    y_pred_4  += model4.predict(test) / (n_splits)
    
    
    print('validation RMSE model 1 fold-',fold+1,': ',rmse[0])
    print('validation RMSE model 2 fold-',fold+1,': ',rmse[1])
    print('validation RMSE model 3 fold-',fold+1,': ',rmse[2])
    print('validation RMSE model 4 fold-',fold+1,': ',rmse[3])
    
    wghts = np.exp(-100*np.array(rmse/sum(rmse)))
    wghts = wghts/sum(wghts)
    
    val_pred           = wghts[0]*val_pred1+wghts[1]*val_pred2+wghts[2]*val_pred3+wghts[3]*val_pred4

    print('validation RMSE fold-',fold+1,': ',np.sqrt(mean_squared_error(y_val, np.exp(val_pred))))
    error.append(np.sqrt(mean_squared_error(y_val, np.exp(val_pred))))
    oof_pred[val_ind]  = np.exp(val_pred)
    print('\n')
    
y_pred_final = np.exp(wghts[0]*y_pred_1+
                      wghts[1]*y_pred_2+
                      wghts[2]*y_pred_3+
                      wghts[3]*y_pred_4
                     )
print('OOF RMSE:- ',np.sqrt(mean_squared_error(oof_pred,train_y)))

validation RMSE model 1 fold- 1 :  2.1135392518618112
validation RMSE model 2 fold- 1 :  2.248138635259805
validation RMSE model 3 fold- 1 :  1.995633933842503
validation RMSE model 4 fold- 1 :  2.150421509140051
validation RMSE fold- 1 :  1.9411797728068534


validation RMSE model 1 fold- 2 :  4.316387393266874
validation RMSE model 2 fold- 2 :  4.433863668282092
validation RMSE model 3 fold- 2 :  4.7339920182402295
validation RMSE model 4 fold- 2 :  4.422532021557629
validation RMSE fold- 2 :  4.338947129128778


validation RMSE model 1 fold- 3 :  2.166778548000724
validation RMSE model 2 fold- 3 :  2.2186414884574557
validation RMSE model 3 fold- 3 :  2.688481280704619
validation RMSE model 4 fold- 3 :  2.3914741068328924
validation RMSE fold- 3 :  2.1222038005570307


validation RMSE model 1 fold- 4 :  1.9983600505354355
validation RMSE model 2 fold- 4 :  2.1015784535099136
validation RMSE model 3 fold- 4 :  2.062255446122924
validation RMSE model 4 fold- 4 :  1.9597519723551424
va

validation RMSE model 1 fold- 32 :  2.194368532745924
validation RMSE model 2 fold- 32 :  1.7931635556915038
validation RMSE model 3 fold- 32 :  1.924468558526987
validation RMSE model 4 fold- 32 :  2.038871336887847
validation RMSE fold- 32 :  1.773755025067972


validation RMSE model 1 fold- 33 :  1.987286563634511
validation RMSE model 2 fold- 33 :  2.202648650601492
validation RMSE model 3 fold- 33 :  2.2412711653047332
validation RMSE model 4 fold- 33 :  2.2163195214789977
validation RMSE fold- 33 :  1.9714919613251256


validation RMSE model 1 fold- 34 :  2.326176900075102
validation RMSE model 2 fold- 34 :  2.3891491408366594
validation RMSE model 3 fold- 34 :  2.2913156762931126
validation RMSE model 4 fold- 34 :  2.380573511974313
validation RMSE fold- 34 :  2.249422146078471


validation RMSE model 1 fold- 35 :  2.4355180381376793
validation RMSE model 2 fold- 35 :  2.418177232224573
validation RMSE model 3 fold- 35 :  2.365308080269771
validation RMSE model 4 fold- 35 :  2.4

validation RMSE model 1 fold- 63 :  2.2546467864868815
validation RMSE model 2 fold- 63 :  2.1498551444085363
validation RMSE model 3 fold- 63 :  2.490186652708036
validation RMSE model 4 fold- 63 :  2.24569220550041
validation RMSE fold- 63 :  2.1219491268202364


validation RMSE model 1 fold- 64 :  2.0736342848991707
validation RMSE model 2 fold- 64 :  2.1838273804093835
validation RMSE model 3 fold- 64 :  2.296103484241993
validation RMSE model 4 fold- 64 :  2.074013313133603
validation RMSE fold- 64 :  2.0016683333836713


validation RMSE model 1 fold- 65 :  2.5164220472065733
validation RMSE model 2 fold- 65 :  2.6206446071803255
validation RMSE model 3 fold- 65 :  2.3916432216126173
validation RMSE model 4 fold- 65 :  2.682736822847257
validation RMSE fold- 65 :  2.381298990767681


validation RMSE model 1 fold- 66 :  2.0195196723592153
validation RMSE model 2 fold- 66 :  2.0919421541593226
validation RMSE model 3 fold- 66 :  1.9804000160970647
validation RMSE model 4 fold- 66 : 

validation RMSE model 1 fold- 94 :  2.259291792488894
validation RMSE model 2 fold- 94 :  2.3117472539186354
validation RMSE model 3 fold- 94 :  2.38193019194235
validation RMSE model 4 fold- 94 :  2.2773049917610306
validation RMSE fold- 94 :  2.161623390681395


validation RMSE model 1 fold- 95 :  2.557592667805985
validation RMSE model 2 fold- 95 :  2.696455412457316
validation RMSE model 3 fold- 95 :  2.450753352054897
validation RMSE model 4 fold- 95 :  2.6123014464599423
validation RMSE fold- 95 :  2.426661950332957


validation RMSE model 1 fold- 96 :  2.58827792254564
validation RMSE model 2 fold- 96 :  2.1929984671429685
validation RMSE model 3 fold- 96 :  2.2502909558064115
validation RMSE model 4 fold- 96 :  2.267402575598449
validation RMSE fold- 96 :  2.1403676316889357


validation RMSE model 1 fold- 97 :  2.0460903240558195
validation RMSE model 2 fold- 97 :  1.9546953363075263
validation RMSE model 3 fold- 97 :  2.1930786025725486
validation RMSE model 4 fold- 97 :  2.1

In [4]:
np.std(error)

0.4859194450382616

In [6]:
submission_df       = pd.read_csv(r'D:\av_hackathons\machine_hack_13\sample_submission.csv')
submission_df['PE'] = y_pred_final
submission_df.to_csv('xgb_blending_within_folds.csv',index=False)